Import libraries

In [63]:
import pandas as pd

Import data

In [69]:
# job titles from rocket
df_test_data = (
    pd.read_csv('job_titles_rocket.gz', delimiter='\t', encoding = "ISO-8859-1", names=['job_title_raw', 'n'])
#     pd.read_csv('job_titles_rocket.gz', delimiter='\t', names=['job_title', 'n'])    
    .dropna()
    .reset_index(drop=True)
)

# transform the data
df_test_data = (
    df_test_data
    .assign(
        job_title_raw=df_test_data['job_title_raw'].str.lower().str.strip()
    )
)

# import assigned roles from thesaurus
df_assigned_roles = (
    pd.read_csv('job_title_dictionary.txt', delimiter='\t')
    .query('FindPhraseStatus == "assignedrole"')
    .drop(columns=['FindPhraseStatus'])
    .reset_index(drop=True)
)

# transform role data
df_assigned_roles = (
    df_assigned_roles
    .assign(
        job_title_raw=df_assigned_roles['FindPhrase'].str.lower().str.strip(),
        job_title=df_assigned_roles['ReplacePhrase'].str.lower().str.strip()
    )
    [['job_title_raw', 'job_title']]
)

Find out exact matches

In [83]:
# first merge
df_exact_matches = (
    df_test_data
    .merge(df_assigned_roles, how='outer', left_on='job_title_raw', right_on='job_title_raw', indicator=True)
)

# split data into matches and non-matches
df_exact_non_matches = (
    df_exact_matches
    .query('_merge == "left_only"')
    .drop(columns=['_merge'])
    .reset_index(drop=True)
)
df_exact_matches = (
    df_exact_matches
    .query('_merge == "both"')
    .drop(columns=['_merge'])
    .reset_index(drop=True)    
)

# display stats
print('Total records: {}'.format(df_test_data.shape[0]))
print('Matched: {}'.format(df_exact_matches.shape[0]))
print('Unmatched: {}'.format(df_exact_non_matches.shape[0]))

Total records: 108153
Matched: 10087
Unmatched: 98066


Define matching function

In [86]:
from thefuzz import process

/usr/local/lib/python3.8/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [84]:
df_exact_matches

,job_title_raw,n,job_title
0,1st grade teacher,48.0,1st grade teacher
1,1st mate,6.0,deckhand
2,2nd grade teacher,52.0,2nd grade teacher
3,2nd mate,9.0,deckhand
4,3d artist,121.0,3d artist
...,...,...,...
10082,zone manager,137.0,zone manager
10083,zone supervisor,7.0,zone manager
10084,zookeeper,36.0,zookeeper
10085,zoologist,7.0,zoologist


In [78]:
df_exact_matches

,job_title_raw,n,job_title,_merge
48,1st grade teacher,48.0,1st grade teacher,both
54,1st mate,6.0,deckhand,both
93,2nd grade teacher,52.0,2nd grade teacher,both
102,2nd mate,9.0,deckhand,both
111,3d artist,121.0,3d artist,both
...,...,...,...,...
108132,zone manager,137.0,zone manager,both
108138,zone supervisor,7.0,zone manager,both
108147,zookeeper,36.0,zookeeper,both
108149,zoologist,7.0,zoologist,both


In [75]:
df_exact_matches['_merge'].unique()

['left_only', 'both', 'right_only']
Categories (3, object): ['left_only', 'right_only', 'both']

In [73]:
df_test_data.shape

(108153, 2)

In [70]:
df_test_data

,job_title_raw,n
0,_,15.0
1,-,307.0
2,--,20.0
3,---,5.0
4,.,374.0
...,...,...
108148,zookeeper intern,6.0
108149,zoologist,7.0
108150,zsm,10.0
108151,zumba fitness instructor,14.0


In [44]:
df_assigned_roles.loc[df_assigned_roles['AssignedRoleRank'] == 1]

,FindPhrase,FindPhraseID,FindPhraseStatus,FindPhraseNote,ReplacePhrase,AssignedRoleID,AssignedRole,AssignedRoleRank,JobTitleUsageCount,JobTitleSearchCount,FindPhraseScore
2,1st Grade Teacher,6,assignedrole,NaN,1st Grade Teacher,9487.0,teacher-firstgrade,1,NaN,NaN,36
8,2nd Grade Teacher,13,assignedrole,NaN,2nd Grade Teacher,9587.0,teacher-secondgrade,1,NaN,NaN,21
13,3D Artist,18,assignedrole,NaN,3D Artist,305.0,artist-3d,1,NaN,NaN,108
14,3D Modeler,20,assignedrole,NaN,3D Modeler,6772.0,modeler-3d,1,NaN,NaN,27
17,3rd Grade Teacher,24,assignedrole,NaN,3rd Grade Teacher,9608.0,teacher-thirdgrade,1,NaN,NaN,53
...,...,...,...,...,...,...,...,...,...,...,...
36412,Zoogler,40053,assignedrole,NaN,Zoogler,10401.0,zoogler,1,NaN,NaN,3
36413,Zookeeper,40054,assignedrole,NaN,Zookeeper,10402.0,zookeeper,1,NaN,NaN,298
36414,Zoologist,40055,assignedrole,NaN,Zoologist,10403.0,zoologist,1,NaN,NaN,330
36416,Zoology Professor,40058,assignedrole,NaN,Zoology Professor,7586.0,professor-zoology,1,NaN,NaN,3
